In [5]:
from collections import Counter 
from glob import iglob
import os
import pathlib
import math 
import re


In [42]:
#Tokenize file
def Tokenizefiles(path):
    tokenizedFile = []
    with open(path,'r') as f :
        for line in f :
            for word in line.split() :
                tokenizedFile.append(word)
    return tokenizedFile

In [43]:
#remove stop words from list of words from tokenize fn.
def RemoveStopWords(listOfWords):
    stop_words = ['.','i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]
    list_preprocessed = [w for w in listOfWords if not w in stop_words]  
    list_preprocessed = []
    for w in listOfWords:  
        if w not in stop_words:  
            list_preprocessed.append(w) 
    return list_preprocessed

In [44]:
def TokenizeQuery(query):
    tokenizedQuery = query.split()
    return tokenizedQuery

#The positional index

In [45]:
# Initialize the dictionary. 
pos_index = {} 

In [46]:
for i in range(1,11): 
    path = "file" + str(i) + ".txt"
    final_token_list =  RemoveStopWords(Tokenizefiles(path)) #tokenize->removeStopWords->ListOfWords without stopWords
    # For position and term in the tokens. 
    for pos, term in enumerate(final_token_list): #EveryWord with its position in list
        # If term already exists in the positional index dictionary. 
        if term in pos_index: 
            # Increment total freq by 1.
            pos_index[term][0] = pos_index[term][0] + 1
            # Check if the term has existed in that DocID before.
            if i in pos_index[term][1]: 
                pos_index[term][1][i].append(pos)
            else: 
                pos_index[term][1][i] = [pos]
        # If term does not exist in the positional index dictionary
        # (first encounter). 
        else:
            # Initialize the list.
            pos_index[term] = []
            #The total frequency is 1.
            pos_index[term].append(1)
            # The postings list is initially empty.
            pos_index[term].append({})
            # Add doc ID to postings list.
            pos_index[term][1][i] = [pos]

In [47]:
print(pos_index)

{'Text': [3, {1: [0], 2: [0], 3: [0]}], 'messages': [3, {1: [1], 2: [1], 3: [1]}], 'used': [3, {1: [2], 2: [2], 3: [2]}], 'personal': [3, {1: [3], 2: [3], 3: [3]}], ',': [5, {1: [4, 6], 2: [4, 6], 5: [8]}], 'family': [1, {1: [5]}], 'social': [3, {1: [7], 2: [7], 5: [9]}], 'purposes': [4, {1: [8], 2: [8], 3: [4], 5: [10]}], 'Governmental': [4, {1: [9], 2: [9], 4: [0], 5: [0]}], 'organizations': [4, {1: [10], 2: [10], 4: [1], 5: [1]}], 'use': [4, {1: [11], 2: [11], 4: [2], 5: [2]}], 'text': [4, {1: [12], 2: [12], 4: [3], 5: [3]}], 'messaging': [4, {1: [13], 2: [13], 4: [4], 5: [4]}], 'communication': [4, {1: [14], 2: [14], 4: [5], 5: [5]}], 'colleagues': [4, {1: [15], 2: [15], 4: [6], 5: [6]}], 'business': [2, {2: [5], 5: [7]}], 'hello': [1, {6: [0]}], 'helwan': [2, {6: [1], 8: [0]}], 'Helwan': [1, {7: [0]}], 'Uni': [1, {7: [1]}], 'many': [1, {7: [2]}], 'university': [2, {8: [1], 10: [1]}], 'cairoo': [1, {9: [0]}], 'cairo': [1, {10: [0]}]}


In [48]:
def PositionalMerge(w1,w2):
    answer = {}
    #i=0 
    #j=0
    wl1 = list(w1[1]) #List of Keys (List of Documents (word1)) #[1,2,3,5,7] [2,3,5,7] [5,7]
    wl2 = list(w2[1]) #List of Keys (List of Documents (word2)) #[1,2,5,7] [2,5,7] [5,7]
    while bool(wl1) and bool(wl2):
        #If Dw1 = Dw2
        if wl1[0] == wl2[0]: #5==5 true
            x = wl1[0] #A document from word1's documents #x=5
            y = wl2[0] #A document from word2's documents #y=5
            pw1 = w1[1][x] # =[0,4] w1[1][2]=[0,3]
            pw2 = w2[1][y] # =[1] w2[1][2]=[1]
            i2 = 0 #
            j2 = 0 #
            while bool(pw1) and bool(pw2): #[0,4] [1]
                if (pw2[0] - pw1[0]) == 1: #true
                    #if D is existing ( just append the index )
                    if x in answer:
                        answer[x].append(pw2[0])
                        pw2.pop(0)
                        pw1.pop(0)
                    #if D is not existing ( append the index and the document )
                    else:
                        answer[x] = [] #answer{1:[],2:[]}
                        answer[x].append(pw2[0]) #answer{1:[1],2:[1]} 
                        pw2.pop(0) #[4]
                        pw1.pop(0) #[]
                    #i2 = i2 + 1
                    #j2 = j2 + 1
                
                elif (pw2[0] - pw1[0]) < 1:
                    j2=j2+1;
                    pw2.pop(0)
                elif (pw2[0] - pw1[0]) > 1:
                    i2=i2+1;
                    pw1.pop(0)
            #i = i+1; #i=1
            #j = j+1; #j=1
            wl1.pop(0)
            wl2.pop(0)
            
        #If Dw1 < Dw2
        elif wl1[0] < wl2[0]:
            #i= i+1;
            wl1.pop(0)
        #If Dw1 > Dw2
        elif wl1[0] > wl2[0]:
            #j=j+1;
            wl2.pop(0)
                    
    return answer                 

In [49]:
print("Enter a phrase query")

Enter a phrase query


In [ ]:
query = input()

In [10]:
preprocessedQuery =  RemoveStopWords(TokenizeQuery(query))

In [11]:
preprocessedQuery

['Governmental', 'organizations']

In [12]:
len(preprocessedQuery)

2

In [13]:
for w in preprocessedQuery:
    print(pos_index[w])

[4, {1: [9], 2: [9], 4: [0], 5: [0]}]
[4, {1: [10], 2: [10], 4: [1], 5: [1]}]


In [14]:
final = PositionalMerge(pos_index[preprocessedQuery[0]],pos_index[preprocessedQuery[1]])
print(final)

{1: [10], 2: [10], 4: [1], 5: [1]}


In [16]:
def tf(path):
    tfs = []
    with open(path,'r') as f :
        for line in f :
            dic={}
        for word in document.split():
            if word in dic:
                dic[word]+=1
            else:
                dic[word]=1
        for word,freq in dic.items():
            print(word,freq)
            dic[word]=freq/len(document.split())
    tfs.append(dic)
    return tfs

In [40]:
def remove_garbage(text):
    text = re.sub(r"\w+",'',text)
    text = text.lower()
    return text

docnum= [ ]
topwords = 100
folderpath = pathlib.Path().absolute()
print(folderpath)
counter = Counter()
for filepath in iglob(os.path.join( folderpath, '*.txt')):
    with open(filepath) as file:
        words = remove_garbage(file.read()).split('  ')
        docnum.append(counter)

listword = [ ]
for word, count in counter.most_common(topwords):
    listword.append(word) #print('{} :{}'.format(count,word))

def word_count(file_name): 
    with open (file_name) as f:
        return(f.read( ).split(' '))

words2 = word_count('F: /files/')

C:\Users\kareem\Desktop\IRPro\IRPro


PermissionError: [Errno 13] Permission denied: 'C:\\Users\\kareem\\Desktop\\IRPro\\IRPro'

In [5]:

uniqewords = set(words).'union(set(words2))

numOfWords = dict. fromkeys(uniqewords, 0) 
for word in words:
    numOfWords[word] += 1

num0fWords2=dict.fromkeys(uniqewords, 0)
 for word in words 2:
    numOfWords 2 [word] += 1

def computeTF(wordDict, bagofWords): 
    tfDict = {}
    bagOfWordsCount=len(bagOfWords)
    for word, count in wordDict.items():
        tfDict[word] = count / float(bagOfWordsCount)
    return tfDict

tf_words = computeTF(numOfWords, words )
tf words2 = compute TF(numOfWords 2, words2)



SyntaxError: EOL while scanning string literal (<ipython-input-5-77b238c12233>, line 3)

In [7]:
def computeIDF (numdocuments):
    N = len(numdocuments)
    idfDict =
    dict.fromkeys(numdocumerits[0].keys(), 0)
    for document in numdocuments: 
         for word, val in document.items( ):
                 if val > 0:
                    idfDict[word]+=1
    for word, val in idfDict.items():
         idfDict[word]= math.log(N /float(val))
    return idfDict

idfs = computeIDF([numOfWords,numofWords2])

def computeTFIDF(tf, idfs)
    tfidf = {} 
    for word, val in tf.items():
        tfidf[word] = val * idfs[word]
    return tfidf

tfidfwords = computeTFIDF(tf words, idfs)
tfidfwords2 = computeTFIDF(tf_words2,idfs)

print(tfidfwords)
print("++"*100) 
print(tfidfwords2)

SyntaxError: invalid syntax (<ipython-input-7-55ca1234b0f6>, line 3)

In [9]:
import re
from collections import Counter 
def cosineSimmilarity(query, document):
    inter = set(query.keys()) & set(document.keys())
    num = sum([query[x] * document[x] for x in inter])
    sum1 = sum([query[x] ** 2 for x in list(query.keys())])
    sum2 = sum([document[x] ** 2 for x in list(document.keys())])
    den = math.sqrt(sum1) * math.sqrt(sum2)
    return float(num) / den

WORD = re.compile(r"\w+")

def textConvert(text):
    words = WORD.findall(text)
    return Counter(words)


query = input("Enter your query")
for i in range(1,11): 
    path = "file" + str(i) + ".txt"
    file = open(path)
    document = file.read().replace("\n", " ")
    file.close()
    userquery = textConvert(query)
    userdocument = textConvert(document)
    sim = cosineSimmilarity(userquery, userdocument)
    print("similarity between query and doucument Number " + str(i) + " is : ", sim)

Enter your query university helwan


similarity between query and doucument Number 1 is :  0.0
similarity between query and doucument Number 2 is :  0.0
similarity between query and doucument Number 3 is :  0.0
similarity between query and doucument Number 4 is :  0.0
similarity between query and doucument Number 5 is :  0.0
similarity between query and doucument Number 6 is :  0.4999999999999999
similarity between query and doucument Number 7 is :  0.0
similarity between query and doucument Number 8 is :  0.9999999999999998
similarity between query and doucument Number 9 is :  0.0
similarity between query and doucument Number 10 is :  0.4999999999999999
